# As voltas de Liskov: iteradores em Python

## A instrução `for`

### Problema

Escreva um programa que exibe os argumentos passados na linha de comando:

```shell
$ ./args açaí banana carambola
./args
açaí
banana
carambola
```

### C

```C
#include <stdio.h>

int main(int argc, char *argv[]) {
    for(int i = 0; i < argc; i++)
        printf("%s\n", argv[i]);
    return 0;
}
```

### Python

```python
import sys

for arg in sys.argv:
    print(arg)
```

### Citação

> Há 2 problemas difíceis em ciência da computação:<br>invalidar cache, nomear coisas e errar por 1.
<br>—*Leon Bambrick*

Duas variantes de *erro-por-1* ao percorrer uma coleção em C:

1. laço termina no penúltimo item, o último é ignorado;
2. laço vai além do último item, acessando
memória inválida ([*segfault*](https://pt.wikipedia.org/wiki/Falha_de_segmenta%C3%A7%C3%A3o)) ou levantando exceção.

A semântica do `for` em Python praticamente elimina esses erros.

## A solução geral: iteráveis e iteradores

### Iteráveis

**Comestível:** que pode ser comida

**Legível:** que pode ser lida

**Iterável:** que pode ser iterada (repetida)

Em programação, **iterável** é uma coleção que pode ser acessada item a item em uma estrutura de iteração.

Java não tinha o conceito de iterável até a versão 5 (2004). Python sempre teve, desde 1991.

### Iterador

Um objeto que implementa uma *interface padrão* para acessar o próximo item de uma coleção.

Em Python, essa interface tem um método: `__next__` (que invocamos através de `next(coleção)`)

## O padrão de projeto Iterator

Fonte: [O Poder dos Geradores (16)](https://speakerdeck.com/ramalho/o-poder-dos-geradores?slide=16)

<img src="iterator-gof.png" style="width: 640px;"/>

<hr/>

Fonte: [O Poder dos Geradores (17)](https://speakerdeck.com/ramalho/o-poder-dos-geradores?slide=17)

<img src="iterator-poster.png" style="width: 640px;"/>

### Implementação "clássica" de Iterator em Python

(código não *pythônico* para ilustrar o padrão do GoF)

In [1]:
class Trem:
    def __init__(self, carros):
        self.carros = carros
    
    def __iter__(self):
        return IterTrem(self.carros)

class IterTrem:
    def __init__(self, carros):
        self.próximo = 0
        self.último = carros - 1
    
    def __next__(self):
        if self.próximo <= self.último:
            self.próximo += 1
            return f'carro #{(self.próximo)}'
        else:
            raise StopIteration()

In [2]:
trem = Trem(3)

for carro in trem:
    print(carro)

carro #1
carro #2
carro #3


### Citação

> Por exemplo, no mundo da orientação a objetos,<br>ouve-se muito sobre "padrões".<br>
> Me pergunto se esses padrões não são, às vezes,<br>evidências [...] do *compilador humano* em ação.<br>
> —*Paul Graham*

Fonte: [Are Design Patterns Missing Language Features?](https://wiki.c2.com/?AreDesignPatternsMissingLanguageFeatures)
(_Será que os padrões de projetos são recursos faltando em uma linguagem?_)

### Implementação pythônica

In [3]:
class Trem:
    def __init__(self, carros):
        self.carros = carros
    
    def __iter__(self):
        for n in range(self.carros):
            yield f'carro #{n}'

In [4]:
trem = Trem(3)

for carro in trem:
    print(carro)

carro #0
carro #1
carro #2


## Como funciona o yield

In [5]:
def gen123():
    ...

In [6]:
def genAB():
    ...

### Reinventando xrange

In [7]:
def zrange(n):
    ...

### Reescrevendo `for` com `while`

In [8]:
iterador = iter('ABC')
while True:
    try:
        item = next(iterador)
    except StopIteration:
        del iterador
        break
    print(item)
        

A
B
C


## A história do yield

Fonte: [Generator Power (11)](https://speakerdeck.com/ramalho/python-generator-power?slide=11)

<img src="liskov-CLU.png" style="width: 640px;"/>

### Homenageadas com o prêmio Turing até 2025

* **Frances E. Allen (2006):** teoria e prática de otimização de compiladores.
* **[Barbara Liskov (2008)](https://amturing.acm.org/award_winners/liskov_1108679.cfm):** fundamentos de linguagens de programação (tipos abstratos) e sistemas distribuídos tolerantes a falha.
* **Shafi Goldwasser (2012):** fundamentos teóricos para a criptografia, incluindo criptografia probabilística.


### A linguagem CLU

A definição de um novo tipo é um *cluster* (semelhante a classe, antes de existir OOP).

> Objetos do subtipo devem se comportar da mesma forma que os do supertipo,<br>
> até onde qualquer programa que use objetos do supertipo possa perceber.<br>
> —*Princípio da substituição de Liskov*

Outras características que influenciaram linguagens atuais:

* exceções com `signal/except`
* atribuição paralela: `x,y,z := f(t)`
* iteradores com `yield`

Fonte: [Generator Power (12)](https://speakerdeck.com/ramalho/python-generator-power?slide=12)

<img src="liskov-yield-for.png" style="width: 640px;"/>

## Iteradores/geradores essenciais em Python

```
iterador ≈ gerador  # sinônimos na comunidade Python
```

Uso _gerador_ quando se trata de séries infinitas. Ex: gerador de números de Fibonacci.

### Como gerar uma série de números?

```C
    for(int i = 0; i < stop; i++)
        ...
```

```python
    for i in range(stop):
        ...
```

### Como numerar uma série de itens?

Vamos reinventar `enumerate`:

In [9]:
def zenum(iterável):
    ...

### Como percorrer duas coleções em paralelo?

Vamos reinventar `zip`:

In [10]:
def zzip(a, b):
    ...

## O pacote `itertools` 👀

https://docs.python.org/pt-br/3.13/library/itertools.html

## Bônus: expressões geradoras

In [11]:
s = 'banana'
g = (ord(c) for c in s)
g

<generator object <genexpr> at 0x7b8c20294fb0>

In [12]:
for l in g:
    print(l)

98
97
110
97
110
97


### Um caso de uso

In [13]:
for vogal in (c for c in s if c not in 'aeiou'):
    print(vogal)

b
n
n


### Trem com genexp

In [14]:
class Trem2:
    def __init__(self, carros):
        self.carros = carros
    
    def __iter__(self):
        return (f'carro #{n}' for n in range(self.carros))

In [15]:
trem = Trem2(3)

iter(trem)

<generator object Trem2.__iter__.<locals>.<genexpr> at 0x7b8c20295630>

In [16]:
for carro in trem:
    print(carro)

carro #0
carro #1
carro #2
